In [50]:
# Librerías
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from astroquery.mpc import MPC

# Cobs API

In [51]:
# Verificar la conexión a internet.
def verificar_conexion():
    try:
        requests.get("http://www.google.com", timeout=5)
        print('✅ Conectado a internet.')
        return True
    
    except requests.ConnectionError:
        print('🛑 Sin conexión a internet.')
        return False

In [52]:
# Conexión con la API de COBS
nombre_cometa =  'C/2017 K2'#'C/2023 A3'
Link_cops_API = f'https://cobs.si/api/obs_list.api?des={nombre_cometa}&format=json&from_date=&to_date=&exclude_faint=False&exclude_not_accurate=False'

if verificar_conexion():
    response = requests.get(Link_cops_API)


if response.status_code == 200:
    content = response.json()
    print('✅ Base de datos actualizada.')

else:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Conectado a internet.
✅ Base de datos actualizada.


In [79]:
# Creación del data frame Cometa
cometa_df = pd.DataFrame(content['objects'])
cometa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2080 entries, 0 to 2079
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       2080 non-null   object 
 1   obs_date                   2080 non-null   object 
 2   comet                      2080 non-null   object 
 3   observer                   2080 non-null   object 
 4   location                   786 non-null    object 
 5   extinction                 120 non-null    object 
 6   obs_method                 2080 non-null   object 
 7   comet_visibility           10 non-null     object 
 8   magnitude                  2078 non-null   object 
 9   conditions                 34 non-null     object 
 10  ref_catalog                2080 non-null   object 
 11  instrument_aperture        2080 non-null   object 
 12  instrument_type            2080 non-null   object 
 13  instrument_focal_ratio     1887 non-null   float

In [80]:
# Sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2080
Variables: 47


In [81]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
87,C,2024-01-29 20:52:48,"{'type': 'C', 'name': 'C/2017 K2', 'fullname':...","{'first_name': 'Steffen', 'last_name': 'Fritsc...",Koeditz,None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,13.1,None,...,None,None,2.0,2.0,s,"DSLR green, mlim=17.7, moon 85% dist 97 deg",None,NaN,,2024-02-12 14:05:12
59,C,2024-02-14 12:43:11,"{'type': 'C', 'name': 'C/2017 K2', 'fullname':...","{'first_name': 'Andrew', 'last_name': 'Pearce'...",None,None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,13.9,None,...,0.02,None,1.6,1.6,s,,None,NaN,,2024-02-15 09:49:44
696,V,2022-07-18 21:20:00,"{'type': 'C', 'name': 'C/2017 K2', 'fullname':...","{'first_name': 'Maciej', 'last_name': 'Kwinta'...",Krzesławice,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,8.8,None,...,None,None,None,None,None,,None,NaN,,2022-08-05 20:08:01
131,V,2024-01-09 10:04:47,"{'type': 'C', 'name': 'C/2017 K2', 'fullname':...","{'first_name': 'Osamu', 'last_name': 'Miyazaki...",None,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,11.1,None,...,None,None,None,None,None,,None,NaN,,2024-01-10 04:56:25
1319,C,2021-09-28 21:07:11,"{'type': 'C', 'name': 'C/2017 K2', 'fullname':...","{'first_name': 'Peter', 'last_name': 'Carson',...",Fegenal de la Sierra;,None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,12.7,None,...,None,None,0.6,0.6,s,,None,NaN,,2022-05-08 07:47:24


In [82]:
# Métodos de observación
cometa_df.obs_method.apply(lambda registro: f'{registro['key']}: {registro['name']}').value_counts()

obs_method
Z: CCD Visual equivalent                                         587
S: In-Out method                                                 568
C: Unfiltered total CCD magnitude                                457
M: Modified-Out method                                           256
k: Kron/Cousins R with CCD                                        83
V: Johnson/Bessel/Kron/Cousins V with CCD                         77
c: Unfiltered nuclear CCD magnitude                                9
B: Simple Out-Out method                                           9
H: Kron/Cousins I with CCD                                         7
A: Pogson                                                          6
I: In-focus                                                        6
D: Johnson/Bessel/Kron/Cousins B with CCD                          5
G: CCD magnitude with a Corion NR-400 "minus-infrared" filter      3
-: Unknown                                                         2
O: Out-of-focus (or ext

In [83]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'])
cometa_df['obs_date'] = pd.to_datetime(pd.to_datetime(cometa_df.obs_date).dt.date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [84]:
# Creación del data frame curva de luz cruda
curva_de_luz_cruda_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']].copy()
curva_de_luz_cruda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2080 entries, 0 to 2079
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  2080 non-null   object        
 1   obs_date        2080 non-null   datetime64[ns]
 2   magnitude       2078 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 48.9+ KB


In [85]:
# Con la información filtrada
filas,columnas = curva_de_luz_cruda_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2080
Variables: 3


In [86]:
# Data Frame de la curva de luz
curva_de_luz_cruda_df.sample(5)

,obs_method_key,obs_date,magnitude
1775,Z,2020-09-12,15.2
1680,S,2021-03-19,14.5
1287,Z,2021-10-09,12.9
1693,Z,2021-02-22,14.4
988,S,2022-06-03,9.4


In [87]:
# Curva de luz cruda.
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_cruda_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# MPC API usando astroquery.

In [88]:
# Creación de data frame Ephemeris (conexión con la API del MPC)
fecha_inicial = curva_de_luz_cruda_df.obs_date.min()
fecha_final = curva_de_luz_cruda_df.obs_date.max()
fechas = (fecha_final - fecha_inicial).days + 1 if (fecha_final - fecha_inicial).days <= 1441 else 1441

ephemeris = MPC.get_ephemeris(nombre_cometa, start = str(fecha_inicial), number = fechas) # type: ignore

ephemeris_df = ephemeris.to_pandas()
ephemeris_df.columns = ephemeris_df.columns.str.lower().str.replace(' ', '_')
ephemeris_df

,date,ra,dec,delta,r,elongation,phase,v,proper_motion,direction
0,2017-06-02,266.594583,64.702222,15.953,16.023,92.2,3.6,19.6,8.85,280.4
1,2017-06-03,266.458333,64.712222,15.947,16.018,92.2,3.6,19.6,8.87,279.3
2,2017-06-04,266.321250,64.721389,15.941,16.012,92.2,3.6,19.6,8.89,278.2
3,2017-06-05,266.183750,64.729167,15.936,16.006,92.2,3.6,19.6,8.91,277.1
4,2017-06-06,266.045417,64.736111,15.930,16.000,92.1,3.6,19.6,8.93,276.0
...,...,...,...,...,...,...,...,...,...,...
1436,2021-05-08,274.442500,42.520556,6.092,6.428,105.0,8.7,13.5,18.28,303.3
1437,2021-05-09,274.302500,42.586667,6.078,6.420,105.4,8.7,13.5,18.47,302.0
1438,2021-05-10,274.158750,42.651111,6.064,6.412,105.7,8.7,13.5,18.67,300.7
1439,2021-05-11,274.011250,42.713611,6.050,6.404,106.1,8.7,13.5,18.86,299.5


In [89]:
# Info del data frame ephemeris
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1441 entries, 0 to 1440
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1441 non-null   datetime64[ns]
 1   ra             1441 non-null   float64       
 2   dec            1441 non-null   float64       
 3   delta          1441 non-null   float64       
 4   r              1441 non-null   float64       
 5   elongation     1441 non-null   float64       
 6   phase          1441 non-null   float64       
 7   v              1441 non-null   float64       
 8   proper_motion  1441 non-null   float64       
 9   direction      1441 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 112.7 KB


In [90]:
# Dar a los datos el formato deseado
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = pd.to_datetime(ephemeris_df.date.dt.date)
ephemeris_df.dtypes

date             datetime64[ns]
ra                      float64
dec                     float64
delta                   float64
r                       float64
elongation              float64
phase                   float64
v                       float64
proper_motion           float64
direction               float64
dtype: object

In [91]:
# Creación del data frame ephemeris filtrada
ephemeris_filtrada_df = ephemeris_df[['date', 'delta','r', 'phase']].copy()
ephemeris_filtrada_df = ephemeris_filtrada_df.rename(columns = {'date':'obs_date'})
ephemeris_filtrada_df

,obs_date,delta,r,phase
0,2017-06-02,15.953,16.023,3.6
1,2017-06-03,15.947,16.018,3.6
2,2017-06-04,15.941,16.012,3.6
3,2017-06-05,15.936,16.006,3.6
4,2017-06-06,15.930,16.000,3.6
...,...,...,...,...
1436,2021-05-08,6.092,6.428,8.7
1437,2021-05-09,6.078,6.420,8.7
1438,2021-05-10,6.064,6.412,8.7
1439,2021-05-11,6.050,6.404,8.7


# Unión de las bases de datos.

In [92]:
# Unión de las bases de datos COBS y MPC
curva_de_luz_procesada_df = curva_de_luz_cruda_df.merge(ephemeris_filtrada_df, on='obs_date')
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase
0,C,2021-05-11,14.2,6.050,6.404,8.7
1,k,2021-05-11,13.4,6.050,6.404,8.7
2,V,2021-05-11,13.8,6.050,6.404,8.7
3,Z,2021-05-11,14.2,6.050,6.404,8.7
4,S,2021-05-08,13.5,6.092,6.428,8.7
...,...,...,...,...,...,...
444,C,2017-06-14,18.4,15.887,15.955,3.6
445,C,2017-06-12,17.8,15.898,15.966,3.6
446,C,2017-06-12,18.7,15.898,15.966,3.6
447,C,2017-06-02,19.1,15.953,16.023,3.6


In [93]:
# Información del data frame curva de lus procesada
curva_de_luz_procesada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  449 non-null    object        
 1   obs_date        449 non-null    datetime64[ns]
 2   magnitude       449 non-null    float64       
 3   delta           449 non-null    float64       
 4   r               449 non-null    float64       
 5   phase           449 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 21.2+ KB


In [94]:
# Reducción de la magnitud aparente
beta = 0

curva_de_luz_procesada_df['magnitud_reducida'] = (
    curva_de_luz_cruda_df['magnitude'] 
    - 5 * np.log10(curva_de_luz_procesada_df['delta'] * curva_de_luz_procesada_df['r'])
    - (beta * curva_de_luz_procesada_df['phase'])
    )
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida
0,C,2021-05-11,14.2,6.050,6.404,8.7,8.258967
1,k,2021-05-11,13.4,6.050,6.404,8.7,7.258967
2,V,2021-05-11,13.8,6.050,6.404,8.7,7.158967
3,Z,2021-05-11,14.2,6.050,6.404,8.7,6.958967
4,S,2021-05-08,13.5,6.092,6.428,8.7,8.635821
...,...,...,...,...,...,...,...
444,C,2017-06-14,18.4,15.887,15.955,3.6,-3.319694
445,C,2017-06-12,17.8,15.898,15.966,3.6,-3.322693
446,C,2017-06-12,18.7,15.898,15.966,3.6,-3.222693
447,C,2017-06-02,19.1,15.953,16.023,3.6,-3.637931


In [95]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='obs_date', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna

In [96]:
# Creación del data frame curva de luz agrupada
curva_de_luz_interna_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').max()
curva_de_luz_interna_df = curva_de_luz_interna_df.reset_index()
curva_de_luz_interna_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida
0,2017-06-02,C,19.1,15.953,16.023,3.6,-3.637931
1,2017-06-12,C,18.7,15.898,15.966,3.6,-3.222693
2,2017-06-14,C,18.4,15.887,15.955,3.6,-3.319694
3,2017-06-16,C,18.5,15.877,15.943,3.7,-2.816692
4,2017-06-17,C,17.8,15.871,15.937,3.7,-3.415054
...,...,...,...,...,...,...,...
305,2021-05-03,Z,13.8,6.163,6.469,8.7,6.196853
306,2021-05-04,Z,12.6,6.149,6.461,8.7,6.104479
307,2021-05-05,Z,13.8,6.134,6.453,8.7,6.012473
308,2021-05-08,Z,13.8,6.092,6.428,8.7,8.635821


In [97]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_interna_df, x=curva_de_luz_interna_df.obs_date, y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Min Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [98]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_interna_df = curva_de_luz_interna_df.copy()
curva_de_luz_interna_df['promedio_movil'] = curva_de_luz_interna_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_interna_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,2017-06-02,C,19.1,15.953,16.023,3.6,-3.637931,NaN
1,2017-06-12,C,18.7,15.898,15.966,3.6,-3.222693,NaN
2,2017-06-14,C,18.4,15.887,15.955,3.6,-3.319694,NaN
3,2017-06-16,C,18.5,15.877,15.943,3.7,-2.816692,NaN
4,2017-06-17,C,17.8,15.871,15.937,3.7,-3.415054,NaN
...,...,...,...,...,...,...,...,...
305,2021-05-03,Z,13.8,6.163,6.469,8.7,6.196853,6.344243
306,2021-05-04,Z,12.6,6.149,6.461,8.7,6.104479,6.256603
307,2021-05-05,Z,13.8,6.134,6.453,8.7,6.012473,6.296439
308,2021-05-08,Z,13.8,6.092,6.428,8.7,8.635821,6.595633


In [99]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_df, x=curva_de_luz_interna_df.obs_date, y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente)

In [100]:
# Creación del data frame curva de luz agrupada
curva_de_luz_externa_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').min()
curva_de_luz_externa_df = curva_de_luz_externa_df.reset_index()
curva_de_luz_externa_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida
0,2017-06-02,C,18.2,15.953,16.023,3.6,-3.637931
1,2017-06-12,C,17.8,15.898,15.966,3.6,-3.322693
2,2017-06-14,C,18.4,15.887,15.955,3.6,-3.319694
3,2017-06-16,C,18.5,15.877,15.943,3.7,-2.816692
4,2017-06-17,C,17.8,15.871,15.937,3.7,-3.415054
...,...,...,...,...,...,...,...
305,2021-05-03,Z,13.8,6.163,6.469,8.7,6.196853
306,2021-05-04,Z,12.6,6.149,6.461,8.7,6.104479
307,2021-05-05,Z,13.8,6.134,6.453,8.7,6.012473
308,2021-05-08,S,13.5,6.092,6.428,8.7,6.135821


In [101]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_externa_df = curva_de_luz_externa_df.copy()
curva_de_luz_externa_df['promedio_movil'] = curva_de_luz_externa_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_externa_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,2017-06-02,C,18.2,15.953,16.023,3.6,-3.637931,NaN
1,2017-06-12,C,17.8,15.898,15.966,3.6,-3.322693,NaN
2,2017-06-14,C,18.4,15.887,15.955,3.6,-3.319694,NaN
3,2017-06-16,C,18.5,15.877,15.943,3.7,-2.816692,NaN
4,2017-06-17,C,17.8,15.871,15.937,3.7,-3.415054,NaN
...,...,...,...,...,...,...,...,...
305,2021-05-03,Z,13.8,6.163,6.469,8.7,6.196853,6.315671
306,2021-05-04,Z,12.6,6.149,6.461,8.7,6.104479,6.256603
307,2021-05-05,Z,13.8,6.134,6.453,8.7,6.012473,6.296439
308,2021-05-08,S,13.5,6.092,6.428,8.7,6.135821,6.238490


In [102]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_externa_df, x=curva_de_luz_externa_df.obs_date, y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Max Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [103]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_df, x=curva_de_luz_externa_df.obs_date, y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Comparación de las curvas de luz

In [104]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_df.obs_date, y=curva_de_luz_externa_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_df.obs_date, y=curva_de_luz_interna_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Promedio Movil')
fig.show()